In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Gradient descent with multiple variables

In [380]:
url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/mpg.csv"
df = pd.read_csv(url)

In [381]:
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino


In [382]:
df_numeric = df.drop(['mpg', 'origin', 'name'], axis=1)
df_numeric['horsepower'].isna().any() # Sprawdzamy czy znajdują się braki w danych
df_numeric = df_numeric.to_numpy()

indices = np.argwhere(np.isnan(df_numeric)).ravel() # Pobieramy indeksy dla brakujących danych
missing_data = df.iloc[indices[::2]]
missing_data

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
32,25.0,4,98.0,NaN,2046,19.0,71,usa,ford pinto
126,21.0,6,200.0,NaN,2875,17.0,74,usa,ford maverick
330,40.9,4,85.0,NaN,1835,17.3,80,europe,renault lecar deluxe
336,23.6,4,140.0,NaN,2905,14.3,80,usa,ford mustang cobra
354,34.5,4,100.0,NaN,2320,15.8,81,europe,renault 18i
374,23.0,4,151.0,NaN,3035,20.5,82,usa,amc concord dl


In [389]:
# Gdy możliwe,uzupełniam braki danych w horsepower, średnią dla danego modelu samochodu
df['horsepower'] = df['horsepower'].fillna(
    df.groupby('name')['horsepower'].transform('mean')
)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
32,25.0,4,98.0,83.4,2046,19.0,71,usa,ford pinto
126,21.0,6,200.0,81.5,2875,17.0,74,usa,ford maverick
330,40.9,4,85.0,NaN,1835,17.3,80,europe,renault lecar deluxe
336,23.6,4,140.0,NaN,2905,14.3,80,usa,ford mustang cobra
354,34.5,4,100.0,NaN,2320,15.8,81,europe,renault 18i
374,23.0,4,151.0,NaN,3035,20.5,82,usa,amc concord dl


In [399]:
x_train = df.drop(['mpg', 'name', 'origin'], axis=1).dropna() # Tworzymy training set
x_train = x_train.reset_index(drop=True).to_numpy()

y_train = df['mpg'].to_numpy()

In [402]:
# Funkcja kosztu
def compute_cost(x, y, w, b):

    total_cost = 0
    m = x.shape[0]
    
    for j in range(m):
        f_wb = (x[j] @ w) + b
        cost = (f_wb - y[j])
        total_cost += cost
        
    avg_cost = total_cost/m
    return avg_cost

In [401]:
w_in = np.array([0.02, 0.02, 0.042, 0.0035, 0.01, 0.02]).reshape(6,1)
b_in = 10

In [403]:
compute_cost(x_train, y_train, w_in, b_in)

array([7.05251472])

In [59]:
def compute_gradient(x, y, w, b):

    m = x.shape[0]
    dj_dw = 0
    dj_db = 0

    for i in range(m):
        f_wb = w * x[i] + b
        dj_dw_i = (f_wb - y[i]) * x[i]
        dj_db_i = f_wb - y[i]
        dj_dw += dj_dw_i
        dj_db += dj_db_i
    dj_dw = dj_dw / m
    dj_db = dj_db / m

    return dj_dw, dj_db

In [60]:
def gradient_descent(x, y, w_in, b_in, alpha, num_iters, gradient_function):

    w = w_in
    b = b_in

    for i in range(num_iters):
        dj_dw, dj_db = gradient_function(x, y, w, b)
        w = w - alpha*dj_dw
        b = b - alpha*dj_db
    return w, b